In [1]:
pokemon_data = read.csv('pokemon.csv')
combat_data = read.csv('combats.csv')
kor_name = read.csv('name_data.csv')

In [2]:
head(pokemon_data)
head(combat_data)
head(kor_name)
nrow(pokemon_data)

,X.,Name,Type.1,Type.2,HP,Attack,Defense,Sp..Atk,Sp..Def,Speed,Generation,Legendary
,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
1,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
3,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
4,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
5,5,Charmander,Fire,,39,52,43,60,50,65,1,False
6,6,Charmeleon,Fire,,58,64,58,80,65,80,1,False


,First_pokemon,Second_pokemon,Winner
,<int>,<int>,<int>
1,266,298,298
2,702,701,701
3,191,668,668
4,237,683,683
5,151,231,151
6,657,752,657


,X,index,kor_name,Name
,<int>,<int>,<chr>,<chr>
1,0,1,이상해씨,Bulbasaur
2,1,2,이상해풀,Ivysaur
3,2,3,이상해꽃,Venusaur
4,3,4,파이리,Charmander
5,4,5,리자드,Charmeleon
6,5,6,리자몽,Charizard


[1] 800

영어이름을 기준으로 한국어이름 열 추가.  
영어이름,세대,전설여부 제거, 중복되는 정보 제거  
타입 원핫인코딩  
각 스탯 정규화  

In [3]:
pokemon_data = merge(pokemon_data,kor_name,by='Name') # 영문명 기준으로 합치기
pokemon_data=pokemon_data[order(pokemon_data$X.),]    # 번호순으로 정렬
pokemon_data = pokemon_data[,c(-1,-11,-12,-13,-14)]   # 불필요 컬럼 제거
names(pokemon_data) = c('index','type1','type2','HP','attack','defense','sp.att','sp.def','speed','name')     #컬럼명 재설정
pokemon_data = pokemon_data[,c(10,1,2,3,4,5,6,7,8,9)]       #컬럼순서 변경
pokemon_data$type1=as.factor(pokemon_data$type1)            #타입을 범주형으로
pokemon_data$type2=as.factor(pokemon_data$type2)
type1_encoded = data.frame(model.matrix(~ type1 -1 ,data = pokemon_data))     #타입 원핫 인코딩
type2_encoded = data.frame(model.matrix(~ type2 -1 ,data = pokemon_data))[-1]
encoding_colnames = c('Bug','Dark','Dragon','Electric','Fairy','Fighting','Fire','Flying','Ghost','Grass','Ground','Ice','Normal','Poison','Psychic','Rock','Steel','Water')
names(type1_encoded) = encoding_colnames
names(type2_encoded) = encoding_colnames
type_encoded = type1_encoded+ type2_encoded  
pokemon_data = pokemon_data[,c(-4,-3)]  #기존의 타입 열 삭제
pokemon_data = cbind(pokemon_data,type_encoded)       #원핫인코드한 열을 추가
row.names(pokemon_data) = NULL                        # 인덱스 제거

In [4]:
nomalize = function(col){
    mean_value = mean(col)
    sd_value = sd(col)
    nomalized_col = (col-mean_value)/sd_value
    return(nomalized_col)
}
min_max_scailing = function(col){
    max_value = max(col)
    min_value = min(col)
    scailed_col = (col-min_value)/(max_value-min_value)
    return(scailed_col)
}

In [5]:
head(pokemon_data)

,name,index,HP,attack,defense,sp.att,sp.def,speed,Bug,Dark,⋯,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,이상해씨,1,45,49,49,65,65,45,0,0,⋯,0,1,0,0,0,1,0,0,0,0
2,이상해풀,2,60,62,63,80,80,60,0,0,⋯,0,1,0,0,0,1,0,0,0,0
3,이상해꽃,3,80,82,83,100,100,80,0,0,⋯,0,1,0,0,0,1,0,0,0,0
4,파이리,5,39,52,43,60,50,65,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,리자드,6,58,64,58,80,65,80,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,리자몽,7,78,84,78,109,85,100,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [6]:
min_max_scailed_data = pokemon_data
nomalized_data = pokemon_data

In [7]:
min_max_scailed_data$HP = min_max_scailing(min_max_scailed_data$HP)
min_max_scailed_data$attack = min_max_scailing(min_max_scailed_data$attack)
min_max_scailed_data$defense = min_max_scailing(min_max_scailed_data$defense)
min_max_scailed_data$sp.att = min_max_scailing(min_max_scailed_data$sp.att)
min_max_scailed_data$sp.def = min_max_scailing(min_max_scailed_data$sp.def)
min_max_scailed_data$speed = min_max_scailing(min_max_scailed_data$speed)

nomalized_data$HP = nomalize(nomalized_data$HP)
nomalized_data$attack = nomalize(nomalized_data$attack)
nomalized_data$defense = nomalize(nomalized_data$defense)
nomalized_data$sp.att = nomalize(nomalized_data$sp.att)
nomalized_data$sp.def = nomalize(nomalized_data$sp.def)
nomalized_data$speed = nomalize(nomalized_data$speed)

In [8]:
head(min_max_scailed_data)
head(nomalized_data)

,name,index,HP,attack,defense,sp.att,sp.def,speed,Bug,Dark,⋯,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,이상해씨,1,0.1732283,0.27500,0.1955556,0.3819444,0.2142857,0.2580645,0,0,⋯,0,1,0,0,0,1,0,0,0,0
2,이상해풀,2,0.2322835,0.35625,0.2577778,0.4861111,0.2857143,0.3548387,0,0,⋯,0,1,0,0,0,1,0,0,0,0
3,이상해꽃,3,0.3110236,0.48125,0.3466667,0.6250000,0.3809524,0.4838710,0,0,⋯,0,1,0,0,0,1,0,0,0,0
4,파이리,5,0.1496063,0.29375,0.1688889,0.3472222,0.1428571,0.3870968,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,리자드,6,0.2244094,0.36875,0.2355556,0.4861111,0.2142857,0.4838710,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,리자몽,7,0.3031496,0.49375,0.3244444,0.6875000,0.3095238,0.6129032,0,0,⋯,0,0,0,0,0,0,0,0,0,0


,name,index,HP,attack,defense,sp.att,sp.def,speed,Bug,Dark,⋯,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,이상해씨,1,-0.8927636,-0.8850274,-0.7321201,-0.1076996,-0.1382553,-0.742106695,0,0,⋯,0,1,0,0,0,1,0,0,0,0
2,이상해풀,2,-0.3106707,-0.4325478,-0.2544730,0.4234838,0.4240576,-0.187462398,0,0,⋯,0,1,0,0,0,1,0,0,0,0
3,이상해꽃,3,0.4654532,0.2635745,0.4278801,1.1317282,1.1738081,0.552063331,0,0,⋯,0,1,0,0,0,1,0,0,0,0
4,파이리,5,-1.1256007,-0.7806090,-0.9368260,-0.2847607,-0.7005682,-0.002580966,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,리자드,6,-0.3882831,-0.3629356,-0.4250612,0.4234838,-0.1382553,0.552063331,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,리자몽,7,0.3878408,0.3331868,0.2572918,1.4504382,0.6114952,1.291589061,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [9]:
name_to_index = function(name){
    index = as.numeric(pokemon_data[pokemon_data$name == name,][2])
    return(index)
}
index_to_name = function(index){
    name = as.character(pokemon_data[pokemon_data$index==index,][1])
    return(name)
}


refine_combat_result = function(index){
    cat(index_to_name(index),'와 싸운 포켓몬들입니다..',index_to_name(index),'가 졌으면 1 이겼으면 0')
    result_df = combat_data[combat_data$First_pokemon==index|combat_data$Second_pokemon==index,]
    result_df[result_df$First_pokemon==index,][1] = 0
    result_df[result_df$Second_pokemon==index,][2] = 0
    result_df[result_df$Winner==index,][3] = 0
    result_df[result_df$Winner!=0,][3]=1
    result_df$index = result_df$First_pokemon+result_df$Second_pokemon
    result_df=result_df[c(-1,-2)]
    result_df=result_df[,c(2,1)]
    row.names(result_df) = NULL
    output = merge(pokemon_data,result_df,by='index')
    output = unique(output)
    row.names(output)=NULL
    return(output) 
}



In [10]:
sample = refine_combat_result(7)

리자몽 와 싸운 포켓몬들입니다.. 리자몽 가 졌으면 1 이겼으면 0

In [11]:
model_by_data = function(data){
    model = glm(formula=Winner~index+name+HP+attack+defense+sp.att+sp.def+speed+Bug+Dark+Dragon+Electric+Fairy+Fighting+Fire+Flying+Ghost+Grass+Ground+Ice+Normal+Poison+Psychic+Rock+Steel+Water,
data=data,family='binomial')
summary(model)
}

In [12]:
model_by_data(sample)

Warning message:
"glm.fit: algorithm did not converge"



Call:
glm(formula = Winner ~ index + name + HP + attack + defense + 
    sp.att + sp.def + speed + Bug + Dark + Dragon + Electric + 
    Fairy + Fighting + Fire + Flying + Ghost + Grass + Ground + 
    Ice + Normal + Poison + Psychic + Rock + Steel + Water, family = "binomial", 
    data = data)

Deviance Residuals: 
  [1]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [26]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [51]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [76]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
[101]  0  0  0  0  0  0  0  0  0

Coefficients: (25 not defined because of singularities)
                 Estimate Std. Error z value Pr(>|z|)
(Intercept)    -2.657e+01  8.075e+05       0        1
index          -1.482e-17  1.684e+03       0        1
name가재장군   -1.909e-14  4.568e+05       0        1
name강철톤     -3.229e-14  5.851e+05       0        1
nam

In [13]:
model = glm(formula=Winner~index+name+HP+attack+defense+sp.att+sp.def+speed+Bug+Dark+Dragon+Electric+Fairy+Fighting+Fire+Flying+Ghost+Grass+Ground+Ice+Normal+Poison+Psychic+Rock+Steel+Water,
data=a,family='binomial')
summary(model)
plot(a$speed,a$Winner)


ERROR: Error in is.data.frame(data): object 'a' not found


포켓몬을 고른다.  
그 포켓몬이 포함된 전투 데이터만 가져온다.  
해당 포켓몬이 이긴 전투의 경우 0, 진 경우 1인 열을 추가한다  
1.해당포켓몬과 전투한 포켓몬의 스탯의 단순 차이(종속변수), 전투결과(독립변수)
2.전투한 포켓몬의 스탯(종속변수) 전투 결과 (독립변수)만 사용

---